In [117]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [118]:
!pip install transformers rouge-score nltk datasets rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
import re
import nltk
import transformers
from datasets import load_dataset, load_metric
import string
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [120]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [121]:
model_checkpoint = "t5-base"
TOKENIZER = AutoTokenizer.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,


In [122]:
path = "/content/gdrive/My Drive/cse_842_project/"

In [123]:
data_files = {"train": path + "bbc_data/train_split.csv", "test": path + "bbc_data/test_split.csv"}
dataset = load_dataset("csv", data_files=data_files)

# DatasetDict({
#    train: Dataset({
#        features: ['filepath', 'class', 'title', 'content',],
#        num_rows: 192368
#    })
# })
# DatasetDict({
#    train: Dataset({
#        features: ['title', 'text', 'url', 'authors', 'timestamp', 'tags'],
#        num_rows: 192368
#    })
# })

  0%|          | 0/2 [00:00<?, ?it/s]

In [124]:
# getting dataset counts
print('Total train data is : {}'.format(len(dataset['train'])))
print('Total test data is : {}'.format(len(dataset['test'])))

Total train data is : 1977
Total test data is : 349


In [125]:
def clean_string(s):
    # add regex to remove urls and bractetted stuff.
    s = re.sub(r'^https?:\/\/.*[\r\n]*', '', s, flags=re.MULTILINE)
    s = re.sub('\s+', ' ', s)
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub('\s+', ' ', s)
    return s

In [126]:
prefix = "summarize: "

max_input_length = 512
max_target_length = 64

def clean_text(text):
    sentences = nltk.sent_tokenize(text.strip())
    sentences_cleaned = [clean_string(s) for sent in sentences for s in sent.split('\n')]
    sentences_cleaned_no_titles = [
        sent for sent in sentences_cleaned
        if len(sent) > 0
    ]
    text_cleaned = " ".join(sentences_cleaned_no_titles)
    return text_cleaned

def preprocess_data(examples, tokenizer=TOKENIZER):
    texts_cleaned = [clean_text(text) for text in examples["content"]]
    inputs = [prefix + text for text in texts_cleaned]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    title_cleaned = [clean_text(text) for text in examples["title"]]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            title_cleaned, max_length=max_target_length, truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [127]:
# datasets_cleaned = dataset.filter(lambda example: (len(example['text']) >= 500) and (len(example['title']) >= 20))
tokenized_dataset = dataset.map(preprocess_data, batched=True)
tokenized_dataset

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


DatasetDict({
    train: Dataset({
        features: ['file_path', 'class', 'title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1977
    })
    test: Dataset({
        features: ['file_path', 'class', 'title', 'content', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 349
    })
})

In [128]:
dataset['train'][1500]

{'file_path': '/media/kuldeep/Work/college_stuff/courses/cse_842/bbc_data/bbc/business/492.txt',
 'class': 'business',
 'title': 'Parmalat bank barred from suing',
 'content': "Bank of America has been banned from suing Parmalat, the food group which went bust in 2003 after an accounting scandal. The bank - along with investors, auditors and the group's managers - wants damages for being a victim of fraud at the hands of the Italian firm. But a judge has barred Bank of America and two auditors from the case. The bank, and Italaudit - formerly the Italian arm of auditor Grant Thornton - face lawsuits and possible prosecution. A second auditor, Deloitte & Touche, has also been banned from the case. Grant Thornton - now rid of the Italian unit at the centre of the case - is still being permitted to sue, as are Consob, Italy's stock market regulator, hundreds of small investors and Parmalat's new managers. Parmalat collapsed in December 2003 after it emerged that the 4bn euros ($5.2bn; £2.

In [129]:
TOKENIZER.convert_ids_to_tokens(tokenized_dataset['train'][1500]['labels'])

['▁par', 'mal', 'at', '▁bank', '▁barre', 'd', '▁from', '▁', 'suing', '</s>']

## Finetuning T5 

In [130]:
!rm -r {model_dir}

rm: cannot remove '/content/gdrive/My': No such file or directory
rm: cannot remove 'Drive/cse_842_project/models/t5/t5-base-title-generation/checkpoint-400': No such file or directory


In [131]:
batch_size = 4
model_name = "t5-base-title-generation"
model_dir = path + 'models/t5/{}'.format(model_name)
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    # report_to="tensorboard"
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [132]:
data_collator = DataCollatorForSeq2Seq(TOKENIZER)

In [133]:
import numpy as np

metric = load_metric("rouge")

def compute_metrics(eval_pred, tokenizer=TOKENIZER):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [134]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=TOKENIZER,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_

In [135]:
trainer.train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-base/snapshots/23aa4f41cb7c08d4b05c8f327b22bfa0eb8c7ad9/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,2.748200,2.104369,34.539700,9.143700,32.211900,32.159300,10.243600
200,2.215400,2.006791,34.456800,10.021100,32.010700,32.011100,9.810900
300,2.149000,1.937868,36.077700,11.210500,33.440100,33.480900,10.318100
400,2.078800,1.887528,36.446900,10.962000,33.955800,34.029400,10.151900
500,2.020400,1.844831,37.224100,11.011900,34.677300,34.649200,9.810900
600,1.719700,1.835135,37.099400,11.338000,34.546400,34.434900,9.790800
700,1.689800,1.814321,37.131600,11.953400,34.588400,34.484600,10.384000
800,1.760800,1.795636,37.732400,12.034900,34.692700,34.656900,10.028700
900,1.712500,1.792261,38.037200,11.971200,35.431900,35.441100,9.957000
1000,1.718800,1.777711,38.334700,12.337300,35.897900,35.861500,9.896800


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: title, file_path, class, content. If title, file_path, class, content are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 349
  Batch size = 4
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: title, file_path, class, content. If title, file_path, class, content are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 349
  Batch size = 4
Saving model checkpoint to /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/checkpoint-200
Configuration saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/checkpoint-200/config.json
Mo

TrainOutput(global_step=1980, training_loss=1.715154597735164, metrics={'train_runtime': 1638.6567, 'train_samples_per_second': 4.826, 'train_steps_per_second': 1.208, 'total_flos': 4752584501345280.0, 'train_loss': 1.715154597735164, 'epoch': 4.0})

In [136]:
trainer.save_model()

Saving model checkpoint to /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation
Configuration saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/config.json
Model weights saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/pytorch_model.bin
tokenizer config file saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/tokenizer_config.json
Special tokens file saved in /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/special_tokens_map.json


In [138]:
model_name = "t5-base-title-generation/checkpoint-1800"
model_dir = path + 'models/t5/{}'.format(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

loading file spiece.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file /content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/checkpoint-1800/config.json
Model config T5Config {
  "_name_or_path": "/content/gdrive/My Drive/cse_842_project/models/t5/t5-base-title-generation/checkpoint-1800",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_n

In [139]:
import torch

# get test split
test_tokenized_dataset = tokenized_dataset["test"]

# # pad texts to the same length
def preprocess_test(examples):
    texts_cleaned = [clean_text(text) for text in examples['content']]
    inputs = [prefix + text for text in texts_cleaned]
    # inputs = [prefix + text for text in examples["content"]]
    model_inputs = tokenizer(
        inputs, max_length=max_input_length, truncation=True, padding="max_length"
    )
    return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=4)
print('dataloader prepared ....')

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
    print(i)
    predictions = model.generate(**batch)
    all_predictions.append(predictions)

print('got predictions ....')

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
title_cleaned = [clean_text(text) for text in test_tokenized_dataset["title"]]

all_titles = tokenizer(
    title_cleaned, max_length=max_target_length,
    truncation=True, padding="max_length"
)["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)

  0%|          | 0/1 [00:00<?, ?ba/s]

dataloader prepared ....
0


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
got predictions ....


{'rouge1': 38.843,
 'rouge2': 13.1288,
 'rougeL': 36.4034,
 'rougeLsum': 36.2537,
 'gen_len': 9.7593}

In [140]:
compute_metrics(predictions_labels)

{'rouge1': 38.843,
 'rouge2': 13.1288,
 'rougeL': 36.4034,
 'rougeLsum': 36.2537,
 'gen_len': 9.7593}

In [141]:
def convert_to_string(input_id):
    tokens = tokenizer.convert_ids_to_tokens(input_id)
    sentence = ''
    for tok in tokens:
        if tok == '</s>':
            break
        elif tok != '<pad>':
            sentence += tok
        else:
            continue
    
    sentence = sentence.replace('▁', ' ').strip()
    return sentence 


In [142]:
convert_to_string(all_titles[0])

'election could be terror target'

In [143]:
true_titles = []
for title in all_titles:
    true_titles.append(convert_to_string(title))


predicted_titles = []
for pre_title in all_predictions_flattened:
    predicted_titles.append(convert_to_string(pre_title))




In [144]:
from rouge import Rouge
rouge = Rouge()

scores = rouge.get_scores(predicted_titles, true_titles, avg=True)

scores

{'rouge-1': {'r': 0.35827534452176324,
  'p': 0.3616307363442033,
  'f': 0.3560570395769102},
 'rouge-2': {'r': 0.12206303724928372,
  'p': 0.12306590257879665,
  'f': 0.12116883263671385},
 'rouge-l': {'r': 0.3405171237549465,
  'p': 0.34425001137035527,
  'f': 0.3386801471283847}}

In [145]:
for t,p in zip(true_titles, predicted_titles):
    print(t)
    print(p)
    print('\n\n')

election could be terror target
terror threat real blair warns



rangers seal old firm win
rangers celtic



s korean lender faces liquidation
lg card to be liquidated if it fails to back bail



hacker threat to apple s itunes
apple s itunes users face security threat



guantanamo man suing government
terror suspect to sue guantanamo



qantas considers offshore option
qantas could move jobs out of australia



tarantino to make friday sequel
tarantino in talks over new film



asia shares defy post quake gloom
tsunamis hit thailand and sri lanka



s korea spending boost to economy
south korea to boost economy



lewsey puzzle over disallowed try
lewsey denied late try



potter director signs warner deal
cuaron signs hollywood deal



air passengers win new eu rights
eu rules to increase compensation



fox attacks blair s tory lies
blair no qualms about lying in election



nhs waiting time target is cut
wales hospital waiting targets drop



henman hopes ended in dubai
henman sl